<a href="https://colab.research.google.com/github/robinhester/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/Copy_of_LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### import
import pandas as pd
import numpy as np
from scipy import stats

In [0]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-02-17 20:50:19--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.1s    

2020-02-17 20:50:20 (138 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [0]:
votes = pd.read_csv('house-votes-84.data', names=['party','handicapped_infants','water_project',
                          'budget','physician_fee_freeze', 'el_salvador_aid',
                          'religious_groups','anti_satellite_ban',
                          'aid_to_contras','mx_missile','immigration',
                          'synfuels', 'education', 'right_to_sue','crime','duty_free',
                          'south_africa'])

votes.head()

,party,handicapped_infants,water_project,budget,physician_fee_freeze,el_salvador_aid,religious_groups,anti_satellite_ban,aid_to_contras,mx_missile,immigration,synfuels,education,right_to_sue,crime,duty_free,south_africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [0]:
votes.dtypes

party                   object
handicapped-infants     object
water-project           object
budget                  object
physician-fee-freeze    object
el-salvador-aid         object
religious-groups        object
anti-satellite-ban      object
aid-to-contras          object
mx-missile              object
immigration             object
synfuels                object
education               object
right-to-sue            object
crime                   object
duty-free               object
south-africa            object
dtype: object

In [0]:
votes = votes.replace({"y": 1, "n": 0, "?": np.NaN})

In [0]:
rep = votes[votes['party']=='republican']

rep.head()

,party,handicapped_infants,water_project,budget,physician_fee_freeze,el_salvador_aid,religious_groups,anti_satellite_ban,aid_to_contras,mx_missile,immigration,synfuels,education,right_to_sue,crime,duty_free,south_africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [0]:
dem = votes[votes['party']=='democrat']

dem.head()

,party,handicapped_infants,water_project,budget,physician_fee_freeze,el_salvador_aid,religious_groups,anti_satellite_ban,aid_to_contras,mx_missile,immigration,synfuels,education,right_to_sue,crime,duty_free,south_africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [0]:
votes.isnull().sum()

party                     0
handicapped_infants      12
water_project            48
budget                   11
physician_fee_freeze     11
el_salvador_aid          15
religious_groups         11
anti_satellite_ban       14
aid_to_contras           15
mx_missile               22
immigration               7
synfuels                 21
education                31
right_to_sue             25
crime                    17
duty_free                28
south_africa            104
dtype: int64

Using hypothesis testing, find an issue that democrats support more than republicans. 

1. Null Hypothesis - Democrats support the issue more than republicans do
2. Alternative Hypothesis - Democrats do not support the issue more than republicans do 
3. Confidence: 95%

In [0]:
#democrats>republicans
stats.ttest_ind(rep.aid_to_contras, dem.aid_to_contras, nan_policy='omit')

Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)

**Conclusion**
Due to the negative t statistic I can safely not reject my null hypothesis and say that democrats supported this issue more than republicans did. 

Using hypothesis testing, find an issue that republicans support more than democrats do. 

1. Null Hypothesis - Republicans support the issue more than democrats do
2. Alternative hypothesis - Republicans do not support the issue more than republicans do
3. Confidence - 95%

In [0]:
#republicans>democrats
stats.ttest_ind(rep.crime, dem.crime, nan_policy='omit')

Ttest_indResult(statistic=16.342085656197696, pvalue=9.952342705606092e-47)

**Conclusion**
Due to the position T Statistic, I can not reject the null hypothesis, republicans support this issue more than democrats do. 

Using hypothesis testing, find an issue that republicans and democrats support equally.

1. Null Hypothesis - Republicans support the issue the same as democrats
2. Alternative hypothesis - Republicans do not support the issue the same as democrats do 
3. Confidence - 95%

In [0]:
#republicans = democrats
stats.ttest_ind(rep.water_project, dem.water_project, nan_policy='omit')

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

**Conclusion** With a T statitic of .08, and a p value of 0.92, I do not reject my null hypothesis, and say that the democrats and republicans support the issue equally. 

In [0]:
##stretch goal Function
def voting (x,y):
  return stats.ttest_ind(x,y,nan_policy='omit')

In [0]:
voting(rep.water_project, dem.water_project)

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

In [0]:
votes.dtypes

party                    object
handicapped_infants     float64
water_project           float64
budget                  float64
physician_fee_freeze    float64
el_salvador_aid         float64
religious_groups        float64
anti_satellite_ban      float64
aid_to_contras          float64
mx_missile              float64
immigration             float64
synfuels                float64
education               float64
right_to_sue            float64
crime                   float64
duty_free               float64
south_africa            float64
dtype: object

In [0]:
voting(rep.handicapped_infants,dem.handicapped_infants)

Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)

In [0]:
voting(rep.budget,dem.budget)

Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)

In [0]:
voting(rep.physician_fee_freeze,dem.physician_fee_freeze)

Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)

In [0]:
voting(dem.el_salvador_aid,rep.el_salvador_aid)

Ttest_indResult(statistic=-21.13669261173219, pvalue=5.600520111729011e-68)

In [0]:
voting(dem.religious_groups, rep.religious_groups)

Ttest_indResult(statistic=-9.737575825219457, pvalue=2.3936722520597287e-20)

In [0]:
voting(dem.anti_satellite_ban, rep.anti_satellite_ban)

Ttest_indResult(statistic=12.526187929077842, pvalue=8.521033017443867e-31)

In [0]:
voting(dem.aid_to_contras, rep.aid_to_contras)

Ttest_indResult(statistic=18.052093200819733, pvalue=2.82471841372357e-54)

In [0]:
voting(dem.mx_missile, rep.mx_missile)

Ttest_indResult(statistic=16.437503268542994, pvalue=5.03079265310811e-47)

In [0]:
voting(rep.immigration, dem.immigration)

Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066)

In [0]:
voting(rep.synfuels, dem.synfuels)

Ttest_indResult(statistic=-8.293603989407588, pvalue=1.5759322301054064e-15)

In [0]:
voting(dem.right_to_sue, rep.right_to_sue)

Ttest_indResult(statistic=-13.51064251060933, pvalue=1.2278581709672758e-34)

In [0]:
voting(dem.duty_free, rep.duty_free)

Ttest_indResult(statistic=12.853146132542978, pvalue=5.997697174347365e-32)

In [0]:
voting(dem.south_africa, rep.south_africa)

Ttest_indResult(statistic=6.849454815841208, pvalue=3.652674361672226e-11)